In [5]:
import numpy
%matplotlib nbagg
from matplotlib import pyplot
from tqdm import tqdm, tnrange

import aotools
from aotools.turbulence import infinitephasescreen_fried
import zernike

# Test the Zernike power spectrum of the infinite phase screen

In [ ]:
n_zerns = 100
nx_size = 128
pxl_scale = 8./nx_size
r0 = 0.2
L0 = 20.
stencil_length_factor = 8

n_scrns = 5000

n_tests = 8
wind_speed = 10 #m/s - just arbitrarily set
time_step = pxl_scale/wind_speed # This is timestep, as one pixel added on each iteration

# Create arrary of zernikes
print("Make Zernikes...")
# Zs = aotools.zernikeArray(n_zerns, nx_size")
Zs = []
for i in range(1, n_zerns+1):
    Zs.append(zernike.anyZernike(int(i), nx_size))
Zs = numpy.array(Zs)
                          
print("Init phase screen")
phase_screen = infinitephasescreen_fried.PhaseScreen(nx_size, pxl_scale, r0, L0, stencil_length_factor=stencil_length_factor)

print("Total Stencil Size: {}m".format(stencil_length_factor*phase_screen.nx_size * pxl_scale))

print("Run tests")
z_coeffs_inf = numpy.zeros((n_tests, n_scrns, n_zerns))
# fig = pyplot.figure()

for n in tnrange(n_tests):
    for i in tnrange(n_scrns):
        
        # Go in all directions
        phase_screen.addRow()
        if n%4 == 0:
            scrn = phase_screen.scrn
        elif n%4 == 1:
            scrn = phase_screen.scrn.T
        elif n%4 == 2:
            scrn = phase_screen.scrn[::-1]
        else:
            scrn = phase_screen.scrn[::-1].T
            
        z_coeffs_inf[n, i] = (scrn * Zs).sum((-1, -2))/(nx_size**2)
    
#         pyplot.cla()
#         pyplot.imshow(scrn)
#         fig.canvas.draw()
    
z_vars_inf = z_coeffs_inf.var(1)


Make Zernikes...
Init phase screen
New size: 129
Total Stencil Size: 64.5m
Run tests



Exception in thread Thread-15:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\site-packages\tqdm\_tqdm.py", line 103, in run
    for instance in self.tqdm_cls._instances:
  File "C:\ProgramData\Anaconda3\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



In [ ]:
pyplot.figure()
pyplot.semilogy(z_vars_inf.T, alpha=0.2)
pyplot.semilogy(z_vars_inf.mean(0), color="k")
pyplot.title("Zernike Power Spectrum")
pyplot.xlabel("Zernike Index")
pyplot.ylabel("Power ($\mathrm{RMS Rad}^2$)")

In [ ]:
pyplot.figure()
plot_zs = [1, 6, 99]
for i, z in enumerate(plot_zs):
    print("plot Z{}".format(z))
    zn_coeffs = z_coeffs_inf.mean(0)[:, z]
    z_ps = abs(numpy.fft.fft(zn_coeffs))**2
    x_vals = numpy.fft.fftfreq(len(z_ps), d=time_step)
    pyplot.loglog(x_vals[:n_scrns//2], z_ps[:n_scrns//2], label="Z_{}".format(z))
    pyplot.xlabel("Frequency (Hz)")
    pyplot.ylabel("Power ($\mathrm{Rad RMS}^2$)")
    pyplot.grid()
pyplot.legend()

# Zernike breakdown of standard FFT screen

In [ ]:
n_zerns = 49
nx_size = 128
pxl_scale = 8./nx_size
r0 = 0.2
L0 = 20.
l0 = 0.01
screen_size_factor = 16

total_scrn_size = screen_size_factor * nx_size
print("Total Screen Size: {}".format(total_scrn_size))

# Create arrary of zernikes
print("Make Zernikes...")
# Zs = aotools.zernikeArray(n_zerns, nx_size")
Zs = []
for i in range(1, n_zerns+1):
    Zs.append(zernike.anyZernike(int(i), nx_size))
Zs = numpy.array(Zs)

wind_speed = 10 #m/s - just arbitrarily set
time_step = pxl_scale/wind_speed # This is timestep, as one pixel added on each iteration
print("Time step: {}s".format(time_step))

n_scrns = total_scrn_size - nx_size

n_tests = 16

z_coeffs_fft = numpy.zeros((n_tests, n_scrns, n_zerns))
z_vars_fft = numpy.zeros((n_tests, n_zerns))
for n in tnrange(n_tests):
    print("Make large phase screen...")
    phase_screen = phasescreen.ft_phase_screen(r0, total_scrn_size, pxl_scale, L0, 0.01)[:, :nx_size]

    print("Get Zernike Coeffs")
#     fig = pyplot.figure()
    for i in tnrange(n_scrns):
        scrn = phase_screen[i:i+nx_size]
        
        if n%4 == 0:
            scrn = scrn
        elif n%4 == 1:
            scrn = scrn.T
        elif n%4 == 2:
            scrn = scrn[::-1]
        else:
            scrn = scrn[::-1].T
        
#         pyplot.cla()
#         pyplot.imshow(scrn)
#         fig.canvas.draw()
        z_coeffs_fft[n, i] = (scrn * Zs).sum((-1, -2))/(Zs[0].sum())

    z_vars_fft[n] = z_coeffs[n].var(0)

In [ ]:
pyplot.figure()
pyplot.semilogy(z_vars_fft.T, alpha=0.2)
pyplot.semilogy(z_vars_fft.mean(0), color="k")
pyplot.title("Zernike Power Spectrum")
pyplot.xlabel("Zernike Index")
pyplot.ylabel("Power ($\mathrm{RMS Rad}^2$)")

In [ ]:
pyplot.figure()
plot_zs = [1, 6, 99]
for i, z in enumerate(plot_zs):
    print("plot Z{}".format(z))
    zn_coeffs = z_coeffs_fft.mean(0)[:, z]
    z_ps = abs(numpy.fft.fft(zn_coeffs))**2
    x_vals = numpy.fft.fftfreq(len(z_ps), d=time_step)
    pyplot.loglog(x_vals[:n_scrns//2], z_ps[:n_scrns//2], label="Z_{}".format(z))
    pyplot.xlabel("Frequency (Hz)")
    pyplot.ylabel("Power ($\mathrm{Rad RMS}^2$)")
    pyplot.grid()
pyplot.legend()